In [3]:
import torch

In [9]:
import torch

def create_graph_coloring_ising_hamiltonian(adj_matrix, k):
    """
    Create an Ising Hamiltonian for the graph coloring problem.
    
    Parameters:
    -----------
    adj_matrix : torch.Tensor
        Adjacency matrix of shape (n, n) where n is the number of nodes.
        adj_matrix[i, j] = 1 if nodes i and j are adjacent, 0 otherwise.
    k : int
        Number of colors.
        
    Returns:
    --------
    torch.Tensor
        Symmetric tensor representing the Ising Hamiltonian of shape (n*k, n*k).
    """
    n = adj_matrix.shape[0]
    
    # Initialize the Hamiltonian matrix
    H = torch.zeros((n*k, n*k), dtype=torch.float)
    
    # Constraint 1: Each node must have exactly one color
    # Add penalty for a node having multiple colors or no color
    for i in range(n):
        for c1 in range(k):
            idx1 = i * k + c1
            
            # Diagonal terms for one-hot constraint
            H[idx1, idx1] = -1
            
            # Cross terms to penalize multiple colors for the same node
            for c2 in range(c1+1, k):
                idx2 = i * k + c2
                H[idx1, idx2] = 2
                H[idx2, idx1] = 2
    
    # Constraint 2: Adjacent nodes cannot have the same color
    # Add penalty for adjacent nodes having the same color
    for i in range(n):
        for j in range(i+1, n):
            if adj_matrix[i, j] > 0:  # If nodes are adjacent
                for c in range(k):
                    idx_i = i * k + c
                    idx_j = j * k + c
                    
                    # Penalty for same color
                    H[idx_i, idx_j] = 2
                    H[idx_j, idx_i] = 2
    
    return H


def encode_graph_coloring_ising(adj_matrix, n_colors, penalty_node=1.0, penalty_edge=1.0):
    n = adj_matrix.shape[0]
    Q = torch.zeros((n * n_colors, n * n_colors), dtype=torch.float32)

    # One-hot color constraint per node
    for i in range(n):
        for c1 in range(n_colors):
            idx1 = i * n_colors + c1
            Q[idx1, idx1] += penalty_node  # X^2 term
            for c2 in range(c1 + 1, n_colors):
                idx2 = i * n_colors + c2
                Q[idx1, idx2] += 2 * penalty_node
                Q[idx2, idx1] += 2 * penalty_node  # ensure symmetry
            Q[idx1, idx1] += -2 * penalty_node  # from -2*X

    # Adjacent nodes with same color
    for i in range(n):
        for j in range(i+1, n):  # upper triangle, symmetric later
            if adj_matrix[i, j] != 0:
                for c in range(n_colors):
                    idx_i = i * n_colors + c
                    idx_j = j * n_colors + c
                    Q[idx_i, idx_j] += penalty_edge
                    Q[idx_j, idx_i] += penalty_edge  # ensure symmetry

    return Q


def decode_solution(x, n, k):
    """
    Decode the solution from the binary vector.
    
    Parameters:
    -----------
    x : torch.Tensor
        Binary vector of shape (n*k,) representing the solution.
    n : int
        Number of nodes.
    k : int
        Number of colors.
        
    Returns:
    --------
    torch.Tensor
        Coloring assignment of shape (n,) where each element is the color index.
    """
    x_reshaped = x.reshape(n, k)
    colors = torch.argmax(x_reshaped, dim=1)
    return colors

def check_solution(adj_matrix, colors):
    """
    Check if the coloring is valid.
    
    Parameters:
    -----------
    adj_matrix : torch.Tensor
        Adjacency matrix of shape (n, n).
    colors : torch.Tensor
        Color assignments of shape (n,).
        
    Returns:
    --------
    bool
        True if the coloring is valid, False otherwise.
    """
    n = adj_matrix.shape[0]
    
    # Check that adjacent nodes have different colors
    for i in range(n):
        for j in range(i+1, n):
            if adj_matrix[i, j] > 0 and colors[i] == colors[j]:
                return False
    
    return True

# Example usage
# def example():
# Create a simple graph with 4 nodes
n = 4
k = 2  # 2 colors

# Create an adjacency matrix for a cycle graph
adj_matrix = torch.tensor(
    [[0, 1, 0, 1], # 2
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0]]
)

# Create the Hamiltonian
J = create_graph_coloring_ising_hamiltonian(adj_matrix, k)
print(f"Hamiltonian shape: {J.shape}")

J2 = encode_graph_coloring_ising(adj_matrix, k)
print(J2)
print(J)


Hamiltonian shape: torch.Size([8, 8])
tensor([[-1.,  2.,  1.,  0.,  0.,  0.,  1.,  0.],
        [ 2., -1.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 1.,  0., -1.,  2.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  2., -1.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0., -1.,  2.,  1.,  0.],
        [ 0.,  0.,  0.,  1.,  2., -1.,  0.,  1.],
        [ 1.,  0.,  0.,  0.,  1.,  0., -1.,  2.],
        [ 0.,  1.,  0.,  0.,  0.,  1.,  2., -1.]])
tensor([[-1.,  2.,  2.,  0.,  0.,  0.,  2.,  0.],
        [ 2., -1.,  0.,  2.,  0.,  0.,  0.,  2.],
        [ 2.,  0., -1.,  2.,  2.,  0.,  0.,  0.],
        [ 0.,  2.,  2., -1.,  0.,  2.,  0.,  0.],
        [ 0.,  0.,  2.,  0., -1.,  2.,  2.,  0.],
        [ 0.,  0.,  0.,  2.,  2., -1.,  0.,  2.],
        [ 2.,  0.,  0.,  0.,  2.,  0., -1.,  2.],
        [ 0.,  2.,  0.,  0.,  0.,  2.,  2., -1.]])


In [23]:
def run_dsb_simulation(J,  steps=1000, a0=1.0, c0=1.0, dt=0.01):
    """
    Run a (dSB) simulation for solving optimization problems.
    
    Args:
        J: Coupling matrix defining the optimization problem
        steps: Number of simulation steps
        a0: System parameter (constant)
        c0: Coupling strength
        dt: Time step size
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
        x_history: History of position values during simulation
    """
    # Get problem size
    N = J.shape[0]
    
    # Initialize positions and momenta
    x = torch.rand(N, dtype=J.dtype, device=J.device) - 0.5  # Position variables
    y = torch.zeros(N, dtype=J.dtype, device=J.device)       # Momentum variables
    
    # # Initialize history arrays
    # x_history = torch.zeros((steps + 1, N), dtype=J.dtype, device=J.device)
    # x_history[0] = x.clone()
    
    # Define a(t) function - linear increase from 0 to a0
    def a_t_func(t):
        return min(a0 * t / (0.2 * steps * dt), a0)
    
    # Main simulation loop
    for step in range(1, steps + 1):
        t = step * dt
        a_t = a_t_func(t)
        
        # First part of symplectic Euler: update momenta
        # ẏ_i = -[a0 - a(t)]x_i + c0 ∑J_ij*x_j
        y -= dt * ((a0 - a_t) * x - c0 * (torch.matmul(J, x)))
        
        # Second part: update positions
        # ẋ_i = a0 * y_i
        x += dt * a0 * y
        
        # Apply inelastic walls: for any |x_i| > 1
        outside_range = torch.abs(x) > 1.0
        if torch.any(outside_range):
            # Replace with sign (±1)
            x[outside_range] = torch.sign(x[outside_range])
            # Set corresponding momenta to 0
            y[outside_range] = 0.0
        
        # Store current positions
        # x_history[step] = x.clone()
    
    # Get binary solution by taking the sign
    binary_solution = x
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j
    ising_energy = -0.5 * torch.sum(torch.matmul(binary_solution, J) * binary_solution)
    
    return binary_solution, ising_energy#, x_history


binary_solution, ising_energy = run_dsb_simulation(J2)

binary_solution


tensor([1., 1., 1., 1., 1., 1., 1., 1.])